In [1]:
print('hello world')

hello world


In [3]:
!pip install pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 32.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 36.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 30.8 MB/s eta 0:00:00
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import pandas as pd
med_qa_data = pd.read_csv('data/MedQA_complete_graded_data.csv', encoding = "ISO-8859-1")

In [9]:
med_qa_data.head()

,index,question,answer,new_IR,IR Grade,new_AR,AR Grade,new_BR,BR Grade,new_DR,...,Rationale.2,IR Grade.1,AR,Rationale.3,AR Grade.1,BR,Rationale.4,BR Grade.1,Unnamed: 29,Unnamed: 30
0,0.0,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,Thrombotic microangiopathy,N,Thrombotic microangiopathy.,N,Thrombotic microangiopathy.,N,Thrombotic microangiopathy,...,"This patient has decreased urinary output, mal...",Y,Cholesterol embolism,The patient's recent history of cardiac cathet...,Y,Contrast-induced nephropathy,This patient's symptoms and clinical history s...,N,NaN,NaN
1,10.0,A 46-year-old man is brought to the emergency ...,"Benzodiazepine intoxication\n""",Alcohol intoxication,N,Alcohol intoxication,N,Alcohol intoxication,N,Alcohol intoxication,...,This patient's symptoms of altered mental stat...,N,Wernicke's encephalopathy,"The patient's symptoms (somnolence, slurred sp...",N,Wernicke's encephalopathy,The prior probability of Wernicke's encephalop...,N,NaN,NaN
2,11.0,A 30-year-old African American woman comes to ...,Histoplasma capsulatum infection,Valley Fever,N,Coccidioidomycosis,N,Histoplasmosis,Y,Fungal infection,...,"This patient presents with fever, cough, chest...",Y,Histoplasmosis,The patient's history of recent hiking in Miss...,Y,Histoplasma capsulatum,The prior probability of Histoplasmosis (a fun...,Y,NaN,NaN
3,16.0,A 67-year-old man who was diagnosed with arthr...,Psoriatic arthritis,Psoriatic arthritis,Y,Psoriatic arthritis,Y,Psoriatic arthritis,Y,Psoriatic arthritis,...,The patient's clinical presentation suggests p...,Y,Psoriatic Arthritis,The patient presents with several features sug...,Y,Psoriatic Arthritis,The prior probability of psoriatic arthritis i...,Y,NaN,NaN
4,17.0,A one-day-old male is evaluated in the hospita...,Duodenal atresia,Gastroschisis,N,Intestinal obstruction,N,Gastroschisis,N,Duodenal atresia,...,This neonate's clinical presentation of biliou...,N,Intestinal atresia,"The symptoms presented by this neonate, such a...",Y,Duodenal atresia in Down Syndrome,"In Bayesian reasoning, we begin with a prior p...",Y,NaN,NaN


In [10]:
med_qa_data.shape

(513, 31)

In [13]:
med_qa_data = med_qa_data.drop(columns = ['index', 'Unnamed: 29', 'Unnamed: 30'])

### The prompts used different reasoning strategies IR = intuitive reasoning, AR = analytic reasoning, BR = Bayesian inference (reasoning), DR = Differential diagnoses (reasoning)

### the correct "golden answer" is given by the "answer" column, and the question posed is in the "question" column

In [14]:
med_qa_data.columns

Index(['question', 'answer', 'new_IR', 'IR Grade', 'new_AR', 'AR Grade',
       'new_BR', 'BR Grade', 'new_DR', 'DR Grade', 'CoT', 'CoT.1', 'GPT-4->',
       'CoT.2', 'Rationale', 'CoT Grade', 'DR ', 'Rationale.1', 'DR Grade.1',
       'IR', 'Rationale.2', 'IR Grade.1', 'AR', 'Rationale.3', 'AR Grade.1',
       'BR', 'Rationale.4', 'BR Grade.1'],
      dtype='object')

In [122]:
# separate out the gpt 3.5 responses from the gpt4 responses - these are all listed after the 'GPT-4->' column
gpt_4_responses = med_qa_data[['question','answer','CoT.2', 'Rationale', 'CoT Grade', 'DR ', 'Rationale.1', 'DR Grade.1',
       'IR', 'Rationale.2', 'IR Grade.1', 'AR', 'Rationale.3', 'AR Grade.1',
       'BR', 'Rationale.4', 'BR Grade.1']]

# the last row is a total row, so leave it out
gpt_4_responses = gpt_4_responses.iloc[0:len(gpt_4_responses) - 1]

In [31]:
gpt_4_responses.head()

,question,answer,CoT.2,Rationale,CoT Grade,DR,Rationale.1,DR Grade.1,IR,Rationale.2,IR Grade.1,AR,Rationale.3,AR Grade.1,BR,Rationale.4,BR Grade.1
0,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,Cholesterol embolization syndrome,This patient's presentation of decreased urina...,Y,Cholesterol embolization syndrome,The differential for this patient includes acu...,Y,Atheroembolic renal disease,"This patient has decreased urinary output, mal...",Y,Cholesterol embolism,The patient's recent history of cardiac cathet...,Y,Contrast-induced nephropathy,This patient's symptoms and clinical history s...,N
1,A 46-year-old man is brought to the emergency ...,"Benzodiazepine intoxication\n""",Alcoholic neuropathy,This patient presents with altered mental stat...,N,Cerebellar disease,The differential for this patient includes alc...,N,Wernicke's encephalopathy,This patient's symptoms of altered mental stat...,N,Wernicke's encephalopathy,"The patient's symptoms (somnolence, slurred sp...",N,Wernicke's encephalopathy,The prior probability of Wernicke's encephalop...,N
2,A 30-year-old African American woman comes to ...,Histoplasma capsulatum infection,Histoplasmosis,This patient has recently been hiking in Missi...,Y,Histoplasmosis,The differential diagnosis in this case includ...,Y,Histoplasmosis,"This patient presents with fever, cough, chest...",Y,Histoplasmosis,The patient's history of recent hiking in Miss...,Y,Histoplasma capsulatum,The prior probability of Histoplasmosis (a fun...,Y
3,A 67-year-old man who was diagnosed with arthr...,Psoriatic arthritis,Psoriatic Arthritis,This patient has a long history of arthritis a...,Y,Psoriatic Arthritis,The differential diagnosis for this patient in...,Y,Psoriatic Arthritis,The patient's clinical presentation suggests p...,Y,Psoriatic Arthritis,The patient presents with several features sug...,Y,Psoriatic Arthritis,The prior probability of psoriatic arthritis i...,Y
4,A one-day-old male is evaluated in the hospita...,Duodenal atresia,Duodenal atresia,The baby in question is displaying signs of in...,Y,Duodenal atresia,"This neonate presents with bilious vomiting, a...",Y,Meconium ileus,This neonate's clinical presentation of biliou...,N,Intestinal atresia,"The symptoms presented by this neonate, such a...",Y,Duodenal atresia in Down Syndrome,"In Bayesian reasoning, we begin with a prior p...",Y


In [34]:
print(gpt_4_responses['question'][100])

A 45-year-old African-American woman comes to the physician after an episode of hemoptysis. Over the past 8 months, she has had a nonproductive cough, fatigue, and a 9-kg (20-lb) weight loss. She drinks a glass of wine every night and does not smoke. She has a history of hypertension and osteoarthritis. She does not take any medications. She has never traveled out of the country. She works as an accountant. Her temperature is 37.0Â°C (98.6Â°F), pulse is 94/min, and blood pressure is 130/90 mm Hg. Lungs are clear to auscultation. An x-ray of the chest shows a 2.5-cm nodule with irregular borders at the apex of the left lung with no mediastinal or hilar enlargement. What is the most likely diagnosis?


In [35]:
# lets use the prompts that they used in the paper and see what happens

ir_prompt = """
Use symptom, signs, and laboratory disease associations to step by step deduce the correct response.
"""

In [51]:
import os
import json

from dotenv import load_dotenv
load_dotenv()

True

In [46]:
from lib.llms.chats import ChatOpenAI

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [84]:
llm_4o = ChatOpenAI(
    base_url='https://api.openai.com/v1',
    model = 'gpt-4o',
    temperature= 1,
    api_key = os.environ.get('OPENAI_API_KEY'),
    max_retries = 4
)

In [85]:
llm_4o('Hello how are you?')

LLMInterface(id_='6e20cf48-6ea2-447a-9185-93fd235180a5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", source=None, channel=None, candidates=["Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"], completion_tokens=29, total_tokens=41, prompt_tokens=12, total_cost=0, logits=[], messages=[AIMessage(id_='6110339a-9a45-4b81-b3fe-7c85cc967bf6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationshi

In [ ]:
qa_template = (
    "Diagnose the patients condition using the given passage with a short factoid answer.\n"
    "Question: {question}\n"
    "Rationale: a step-by-step deduction that identifies the correct response. {prompt}"
               )

In [62]:
qa_template

'{instructions}\n{question}'

In [68]:
from lib.llms import PromptTemplate


prompt_template = PromptTemplate(qa_template)

In [65]:
print(prompt_template.populate(instructions = "these are the instructions", question = "this is my special question"))

these are the instructions
this is my special question


In [69]:


print(prompt_template.populate(question = "this is my special question", prompt = ir_prompt))

Diagnose the patients condition using the given passage with a short factoid answer.
Question:this is my special question
Rationale: a step-by-step deduction that identifies the correct response. 
Use symptom, signs, and laboratory disease associations to step by step deduce the correct response.



In [73]:
q = gpt_4_responses['question'][0]

In [74]:
print(prompt_template.populate(question = q, prompt = ir_prompt))

Diagnose the patients condition using the given passage with a short factoid answer.
Question:Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise. He has type 2 diabetes mellitus and osteoarthritis of the hips. Prior to admission, his medications were insulin and naproxen. He was also started on aspirin, clopidogrel, and metoprolol after the coronary intervention. His temperature is 38Â°C (100.4Â°F), pulse is 93/min, and blood pressure is 125/85 mm Hg. Examination shows mottled, reticulated purplish discoloration of the feet. Laboratory studies show:
Hemoglobin count 14 g/dL
Leukocyte count 16,400/mm3
Segmented neutrophils 56%
Eosinophils 11%
Lymphocytes 31%
Monocytes 2%
Platelet count 260,000/mm3
Erythrocyte sedimentation rate 68 mm/h
Serum
Urea nitrogen 25 mg/dL
Creatinine 4.2 mg/dL
Renal biopsy shows intravascular spindle-shaped vacuoles. What is the most likely cause of 

In [86]:
res = llm_4o(prompt_template.populate(question = q, prompt = ir_prompt))

In [89]:
print(res.text)

The patient's condition is most likely due to cholesterol embolization syndrome (also known as atheroembolism). Here is a step-by-step explanation:

1. **Recent History and Symptoms**: The patient underwent cardiac catheterization and stenting two weeks ago. Following this, he's experiencing decreased urinary output and malaise. The reduced kidney function suggests possible renal injury.

2. **Skin Findings**: Mottled, reticulated purplish discoloration of the feet indicates livedo reticularis, which is commonly associated with embolic events.

3. **Laboratory Findings**: 
   - Elevated eosinophils (11%) suggesting an eosinophilic response, which is classic for cholesterol emboli.
   - Elevated leukocyte count and sedimentation rate indicate an inflammatory process.

4. **Renal Findings**: Elevated serum creatinine (4.2 mg/dL) and urea nitrogen (25 mg/dL) indicate acute kidney injury, a common occurrence in cholesterol embolization.

5. **Renal Biopsy**: The presence of intravascular s

In [172]:
from json import JSONDecodeError
from collections import defaultdict
from lib.reasoning.pipelines.answer_with_context import AnswerWithContextPipeline
from itertools import islice
import time
import asyncio
from typing import Union
import numpy as np


async def async_get_extraction(answer_with_context_pipeline: AnswerWithContextPipeline, prompt: str, evidence: str, history = []):
    try:
        extracted_answer = await answer_with_context_pipeline.ainvoke(question = prompt, evidence = evidence, history = history)

        return extracted_answer
    except Exception as e:
        print(e)
        return {"explanation": "Failed to parse",  'reason': e}

def batch_tasks(iterable, batch_size):
    iterator = iter(iterable)
    while True:
        batch = list(islice(iterator, batch_size))
        if not batch:
            break
        yield batch


async def async_extract(answer_with_context_pipeline: AnswerWithContextPipeline, prompt, evidence_arr, batch_size = 100, history = []):
    n_data = len(evidence_arr)
    
    tasks = []
    for i, evidence in enumerate(evidence_arr):
        print(f'====== \n processing {i + 1}/{n_data} files \n')

        tasks.append(async_get_extraction(answer_with_context_pipeline, prompt, evidence, history = history))

    extracted_answers = []
    needs_reprocessing = []
    for batch_number, batch in enumerate(batch_tasks(tasks, batch_size)):
        print(f'processing batch {batch_number} of size {batch_size}')
        settled_tasks = await asyncio.gather(*batch)
        for i,initial_extraction in enumerate(settled_tasks):
            idx = batch_number*batch_size + i
            if isinstance(initial_extraction, dict):
                needs_reprocessing.append(initial_extraction)
                extracted_answers.append(initial_extraction)
            else:
                try:
                    parsed_initial_extraction = json.loads(initial_extraction.content)

                    extracted_answers.append(parsed_initial_extraction)

                except JSONDecodeError as e:
                    needs_reprocessing.append({"answer": "Failed to parse", "idx": idx, 'reason': e})
                    extracted_answers.append(initial_extraction.content)
        # avoid rate limit
        time.sleep(6)
        
    return extracted_answers, needs_reprocessing

In [97]:
print('Follow the following format.\n\nQuestion: ${the question to be answered}\nRationale: a step-by-step deduction that identifies the correct response. ${Provide a step by step deduction that identifies the correct response.}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'
)

Follow the following format.

Question: ${the question to be answered}
Rationale: a step-by-step deduction that identifies the correct response. ${Provide a step by step deduction that identifies the correct response.}
Answer: ${a short factoid answer, often between 1 and 5 words}


In [98]:
qa_template = (
    "Question: {context}\n"
    "Rationale: a step-by-step deduction that identifies the correct response. {question}"
    "Answer:"
    )

llm_4o = ChatOpenAI(
    base_url='https://api.openai.com/v1',
    model = 'gpt-4o',
    temperature= 1,
    api_key = os.environ.get('OPENAI_API_KEY'),
    max_retries = 4
)


answer_pipeline = AnswerWithContextPipeline(
    llm = llm_4o,
    qa_template = qa_template
    # system_prompt="you could put a system prompt here"
)

In [100]:
# lets put together the list of questions we want to ask

questions = gpt_4_responses['question'].values

In [101]:
ir_prompt = """
Use symptom, signs, and laboratory disease associations to step by step deduce the correct response.
"""

In [ ]:
answers, need_reprocess = await async_extract(answer_pipeline, ir_prompt, questions, batch_size = 100)

 processing 1/513 files 

 processing 2/513 files 

 processing 3/513 files 

 processing 4/513 files 

 processing 5/513 files 

 processing 6/513 files 

 processing 7/513 files 

 processing 8/513 files 

 processing 9/513 files 

 processing 10/513 files 

 processing 11/513 files 

 processing 12/513 files 

 processing 13/513 files 

 processing 14/513 files 

 processing 15/513 files 

 processing 16/513 files 

 processing 17/513 files 

 processing 18/513 files 

 processing 19/513 files 

 processing 20/513 files 

 processing 21/513 files 

 processing 22/513 files 

 processing 23/513 files 

 processing 24/513 files 

 processing 25/513 files 

 processing 26/513 files 

 processing 27/513 files 

 processing 28/513 files 

 processing 29/513 files 

 processing 30/513 files 

 processing 31/513 files 

 processing 32/513 files 

 processing 33/513 files 

 processing 34/513 files 

 processing 35/513 files 

 processing 36/513 files 

 processing 37/513 files 

 processin

/Users/benjamindykstra/development/health-tech-llm-code-lab/venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
/Users/benjamindykstra/development/health-tech-llm-code-lab/venv/lib/python3.10/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide


processing batch 1 of size 100
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 i

In [167]:
for ans in answers[:5]:
    print(ans)
    print('==============\n')

To determine the cause of this patient's symptoms, let's approach the problem step by step by analyzing the key symptoms, signs, laboratory findings, and the patient's history.

1. **Patient History and Procedures:**
   - The patient, a 61-year-old man with a history of type 2 diabetes mellitus and osteoarthritis, underwent an emergency cardiac catheterization with stenting for unstable angina pectoris two weeks ago.
   - Post-procedure, he was started on aspirin, clopidogrel, and metoprolol.

2. **Current Symptoms and Physical Examination:**
   - The patient presents with decreased urinary output and malaise.
   - Examination reveals mottled, reticulated purplish discoloration of the feet, which is indicative of livedo reticularis, often associated with embolic phenomena.

3. **Laboratory Findings:**
   - Leukocyte count is elevated at 16,400/mm³ with a notable eosinophilia (11%), which can suggest an allergic reaction or systemic inflammation.
   - Erythrocyte sedimentation rate (ESR

# Lets create an eval harness

In [170]:
def eval_study_responses(response_df, true_col, pred_col):
    # super simple, is answer in prediction
    no_nulls = response_df[~response_df[pred_col].isna()].reset_index()
    true_answers = no_nulls[true_col]
    pred_answers = no_nulls[pred_col]

    correct = pd.Series([a.lower() in b.lower() for a, b in zip(true_answers, pred_answers)])

    print(f'{pred_col} # correct = {correct.sum()} / {len(correct)}')

    return no_nulls[correct][[true_col, pred_col]], no_nulls[~correct][[true_col, pred_col]]


def eval_responses(response_df, true_col, llm_res):
    no_nulls = response_df[~response_df[true_col].isna()].reset_index()
    true_answers = no_nulls[true_col]
    

    correct = pd.Series([a.lower() in b.lower() for a, b in zip(true_answers, llm_res)])

    print(f'# correct = {correct.sum()} / {len(correct)}')

    # return no_nulls[correct][[true_col, pred_col]], no_nulls[~correct][[true_col, pred_col]]

In [168]:
gpt_4_responses.columns

Index(['question', 'answer', 'CoT.2', 'Rationale', 'CoT Grade', 'DR ',
       'Rationale.1', 'DR Grade.1', 'IR', 'Rationale.2', 'IR Grade.1', 'AR',
       'Rationale.3', 'AR Grade.1', 'BR', 'Rationale.4', 'BR Grade.1'],
      dtype='object')

In [171]:
study_answer_cols = ['CoT.2', 'DR ',
       'IR', 'AR','BR']

for col in study_answer_cols:

    correct, incorrect = eval_study_responses(gpt_4_responses, 'answer', col)

CoT.2 # correct = 186 / 498
DR  # correct = 209 / 497
IR # correct = 197 / 497
AR # correct = 197 / 497
BR # correct = 171 / 496


In [166]:
eval_responses(gpt_4_responses, 'answer', answers)

# correct = 256 / 512


In [173]:
dr_prompt = """
Use step by step deduction to create a differential diagnosis and then use step by step deduction to determine the correct response.
"""
dr_answers, need_reprocess = await async_extract(answer_pipeline, dr_prompt, questions, batch_size = 100)

 processing 1/513 files 

 processing 2/513 files 

 processing 3/513 files 

 processing 4/513 files 

 processing 5/513 files 

 processing 6/513 files 

 processing 7/513 files 

 processing 8/513 files 

 processing 9/513 files 

 processing 10/513 files 

 processing 11/513 files 

 processing 12/513 files 

 processing 13/513 files 

 processing 14/513 files 

 processing 15/513 files 

 processing 16/513 files 

 processing 17/513 files 

 processing 18/513 files 

 processing 19/513 files 

 processing 20/513 files 

 processing 21/513 files 

 processing 22/513 files 

 processing 23/513 files 

 processing 24/513 files 

 processing 25/513 files 

 processing 26/513 files 

 processing 27/513 files 

 processing 28/513 files 

 processing 29/513 files 

 processing 30/513 files 

 processing 31/513 files 

 processing 32/513 files 

 processing 33/513 files 

 processing 34/513 files 

 processing 35/513 files 

 processing 36/513 files 

 processing 37/513 files 

 processin

/Users/benjamindykstra/development/health-tech-llm-code-lab/venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
/Users/benjamindykstra/development/health-tech-llm-code-lab/venv/lib/python3.10/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide


processing batch 1 of size 100
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 i

In [174]:
eval_responses(gpt_4_responses, 'answer', dr_answers)

# correct = 259 / 512
